In [1]:
# Import data from Excel sheet
import pandas as pd
df = pd.read_excel('hippocampus_volume_relevance_DELCODE.xlsx', sheet_name='DELCODE_LRP_CMP')
#print(df)
sid = df['SID']
grp = df['prmdiag']
age = df['age']
sex = df['sex_bin_1female']
tiv = df['TIV_CAT12']
field = df['FieldStrength']
amybin = df['ratio_Abeta42_40_pos']
grpbin = (grp > 0) # 0=CN, ...

In [2]:
# load sample scan from CN class
import glob
import numpy as np
import nibabel as nib
# define FOV to reduce required memory size
x_range_from = 10; x_range_to = 110
y_range_from = 13; y_range_to = 133
z_range_from = 5; z_range_to = 105
fname = glob.glob('mwp1_CAT12_DELCODE/0_CN/mwp1mprage_ca8cd6a27M0_T1_01_1.nii*')[0]
nifti = nib.load(fname)
data = nifti.get_fdata()
# prepare image
img = data[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to]
img = np.transpose(img, (2, 0, 1))  # reorder dimensions to match coronal view z*x*y in MRIcron etc.
img = np.flip(img)  # flip all positions
img = np.expand_dims(img, axis=(0,4))
print(img.shape)

(1, 100, 100, 120, 1)


In [3]:
# Match covariate information
import re
import numpy as np
from keras.utils import to_categorical
debug = False
cov_idx = [-1]
print('Matching covariates for loaded files ...')
for i,id in enumerate(sid):
  p = [j for j,x in enumerate(fname) if re.search('_%s' % id, x)] # extract ID numbers from filename, translate to Excel row index
  if len(p)==0:
    if debug: print('Did not find %04d' % id) # did not find Excel sheet subject ID in loaded file selection
  else:
    if debug: print('Found %04d in %s: %s' % (id, p[0], fname))
    cov_idx[p[0]] = i # store Excel index i for data file index p[0]
print('Checking for scans not found in Excel sheet: ', sum(x<0 for x in cov_idx))

labels = pd.DataFrame({'Group':grpbin}).iloc[cov_idx, :]
labels = to_categorical(np.asarray(labels)) # use grps to access original labels
grps = pd.DataFrame({'Group':grp, 'RID':sid}).iloc[cov_idx, :]


# Perform regression-based covariates cleaning
from sklearn import linear_model
from pandas import DataFrame

covariates = DataFrame({'Age':age, 'Sex':sex, 'TIV':tiv, 'FieldStrength':field}).iloc[cov_idx, :]
print(covariates.head())
covariates = covariates.to_numpy(dtype=np.float32) # convert dataframe to nparray with 32bit types

Using TensorFlow backend.


Matching covariates for loaded files ...
Checking for scans not found in Excel sheet:  1
      Age  Sex   TIV  FieldStrength
473  75.6    0  1446              3


In [4]:
# load linear model coefficients
import h5py
hf = h5py.File('linear_models_ADNI2.hdf5', 'r')
hf.keys  # read keys
lmarray = np.array(hf.get('linearmodels'), dtype=np.float32)  # stores 4 coefficients + 1 intercept per voxel
hf.close()

In [5]:
from sklearn import linear_model

residuals = np.copy(img)
# create a second version of the residuals for lowered itensities (occluded image)
residuals_occluded = np.multiply(img, 0.5) # reduce local intensity/volume by 50%, i.e. 0.46 -> 0.23

lm = linear_model.LinearRegression()
for k in range(img.shape[3]):
    if (k % 10 == 0): print('residualizing slice z=', str(k))
    for j in range(img.shape[2]):
        for i in range(img.shape[1]):
            if any(lmarray[k, j, i, :] != 0): # skip empty voxels/space
                # load fitted linear model coefficients
                lm.coef_ = lmarray[k, j, i, :4]
                lm.intercept_ = lmarray[k, j, i, 4]
                pred = lm.predict(covariates)  # calculate prediction for all subjects
                residuals[0, i, j, k, 0] = residuals[0, i, j, k, 0] - pred  # % subtract effect of covariates from original values (=calculate residuals)
                residuals_occluded[0, i, j, k, 0] = residuals_occluded[0, i, j, k, 0] - pred

residualizing slice z= 0
residualizing slice z= 10
residualizing slice z= 20
residualizing slice z= 30
residualizing slice z= 40
residualizing slice z= 50
residualizing slice z= 60
residualizing slice z= 70
residualizing slice z= 80
residualizing slice z= 90
residualizing slice z= 100
residualizing slice z= 110


In [6]:
# Display a single scan (residuals)
from matplotlib import pyplot as plt
#import numpy as np
test_img = residuals[0, :,:,:, 0] # img[0, :,:,:, 0]
ma = np.max(test_img)
mi = np.min(test_img)
test_img = (test_img - mi) / (ma - mi) # normalising to (0-1) range
#test_img = (test_img - test_img.mean())/test_img.std() # normalizing by mean and sd
print('displaying residual image ', fname)
for i in range(test_img.shape[2]):
  if (i % 25 == 0): # only display each xxth slice
    plt.figure()
    a = test_img[:,:,i]
    plt.imshow(a, cmap='gray')

displaying residual image  mwp1_CAT12_DELCODE/0_CN\mwp1mprage_ca8cd6a27M0_T1_01_1.nii.gz


In [7]:
# Load CNN model from disk
# specify version of tensorflow
#%tensorflow_version 1.x  # <- use this for Google colab
import tensorflow as tf
# downgrade to specific version
#!pip install tensorflow-gpu==1.15
#import tensorflow as tf
print(tf.__version__)

# disable tensorflow deprecation warnings
import logging
logging.getLogger('tensorflow').disabled=True

1.15.4


In [8]:
from keras.models import load_model, Model
import innvestigate
import innvestigate.utils as iutils
import os

mymodel = load_model('model_checkpoints/resmodel_wb_whole_ds.hdf5')

# see https://github.com/albermax/innvestigate/blob/master/examples/notebooks/imagenet_compare_methods.ipynb for a list of alternative methods
methods = [ # tuple with method,     params,                  label
#            ("deconvnet",            {},                      "Deconvnet"),
#            ("guided_backprop",      {},                      "Guided Backprop"),
#            ("deep_taylor.bounded",  {"low": -1, "high": 1},  "DeepTaylor"),
#            ("input_t_gradient",     {},                      "Input * Gradient"),
#            ("lrp.z",                {},                      "LRP-Z"),
#            ("lrp.epsilon",          {"epsilon": 1},          "LRP-epsilon"),
#            ("lrp.alpha_1_beta_0",   {"neuron_selection_mode":"index"},     "LRP-alpha1beta0"),
        ("lrp.sequential_preset_a", {"neuron_selection_mode": "index", "epsilon": 1e-10}, "LRP-CMPalpha1beta0"), # LRP CMP rule taken from https://github.com/berleon/when-explanations-lie/blob/master/when_explanations_lie.py
]

#model_wo_softmax = iutils.keras.graph.model_wo_softmax(mymodel)  ## sometimes raises: ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.
#model_wo_softmax = Model(inputs=mymodel.inputs,
#                          outputs=iutils.keras.graph.pre_softmax_tensors(mymodel.outputs),
#                          name=(mymodel.name + '_wo_softmax')) 
#model_wo_softmax.summary()
mymodel.layers[-1].activation=tf.keras.activations.linear
mymodel.save('tmp_wo_softmax.hdf5')
model_wo_softmax = load_model('tmp_wo_softmax.hdf5')
os.remove('tmp_wo_softmax.hdf5')
model_wo_softmax.summary()

# create analyzer
analyzers = []
for method in methods:
    analyzer = innvestigate.create_analyzer(method[0], model_wo_softmax, **method[1])
    # Some analyzers require training.
    #   analyzer.fit(test_img, batch_size=20, verbose=1)
    #  analyzers.append(analyzer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 100, 100, 120, 5)  140       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 50, 60, 5)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 60, 5)     20        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 60, 5)     680       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 25, 25, 30, 5)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 25, 30, 5)     20        
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 25, 25, 30, 5)     680       
__________

In [9]:
# define resolution for occlusion analysis
step_size = 3 # should be an odd number
half_step_size = np.floor(step_size/2).astype(int)
radius = 10 # evaluate 15, 10, 5

In [12]:
prediction_map = np.zeros(img.shape, dtype=np.float32) # store class probabilities for MCI/AD
pos_relevance_map = np.zeros(img.shape, dtype=np.float32) # store total relevance for MCI/AD
neg_relevance_map = np.zeros(img.shape, dtype=np.float32) # store total relevance for MCI/AD

for k in range(half_step_size, img.shape[3]-half_step_size, step_size):
    print('processing slice ', str(k+1), ' of ', str(img.shape[3]))
    for j in range(half_step_size, img.shape[2]-half_step_size, step_size):
        for i in range(half_step_size, img.shape[1]-half_step_size, step_size):
            fov_x_start = max(0,i-radius)
            fov_x_end = min(i+radius, img.shape[1])
            fov_y_start = max(0,j-radius)
            fov_y_end = min(j+radius, img.shape[2])
            fov_z_start = max(0,k-radius)
            fov_z_end = min(k+radius, img.shape[3])
            
            new_data = np.copy(residuals)
            new_data[0, fov_x_start:fov_x_end, 
                      fov_y_start:fov_y_end ,
                      fov_z_start:fov_z_end, 0] = residuals_occluded[0, fov_x_start:fov_x_end, 
                                                      fov_y_start:fov_y_end ,
                                                      fov_z_start:fov_z_end, 0]
            
            prob = mymodel.predict(new_data)[0,1] # only store prob of AD/MCI
            tmp = np.full([step_size, step_size, step_size], prob)
            prediction_map[0, (i-half_step_size):(i+half_step_size+1), 
                           (j-half_step_size):(j+half_step_size+1), 
                           (k-half_step_size):(k+half_step_size+1),
                           0] = tmp
            
            a = analyzer.analyze(new_data, neuron_selection=1)
            apos = np.maximum(a, 0) # remove negative relevance
            tmp = np.full([step_size, step_size, step_size], np.sum(apos))
            pos_relevance_map[0, (i-half_step_size):(i+half_step_size+1), 
                           (j-half_step_size):(j+half_step_size+1), 
                           (k-half_step_size):(k+half_step_size+1),
                           0] = tmp
            aneg = np.minimum(a, 0) # remove positive relevance
            tmp = np.full([step_size, step_size, step_size], np.sum(aneg))
            neg_relevance_map[0, (i-half_step_size):(i+half_step_size+1), 
                           (j-half_step_size):(j+half_step_size+1), 
                           (k-half_step_size):(k+half_step_size+1),
                           0] = tmp

processing slice  2  of  120
processing slice  5  of  120
processing slice  8  of  120
processing slice  11  of  120
processing slice  14  of  120
processing slice  17  of  120
processing slice  20  of  120
processing slice  23  of  120
processing slice  26  of  120
processing slice  29  of  120
processing slice  32  of  120
processing slice  35  of  120
processing slice  38  of  120
processing slice  41  of  120
processing slice  44  of  120
processing slice  47  of  120
processing slice  50  of  120
processing slice  53  of  120
processing slice  56  of  120
processing slice  59  of  120
processing slice  62  of  120
processing slice  65  of  120
processing slice  68  of  120
processing slice  71  of  120
processing slice  74  of  120
processing slice  77  of  120
processing slice  80  of  120
processing slice  83  of  120
processing slice  86  of  120
processing slice  89  of  120
processing slice  92  of  120
processing slice  95  of  120
processing slice  98  of  120
processing sl

In [16]:
# save output as nifti images

a = np.copy(prediction_map)
a = np.squeeze(a) # remove first subject index and last color channel again
a = np.flip(a) # flip all positions
a = np.transpose(a, (1, 2, 0)) # reorder dimensions from coronal view z*x*y back to x*y*z
new_data = np.zeros(data.shape, dtype=np.float32)
new_data[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to] = a
out_nifti_pm = nib.Nifti1Image(new_data, nifti.affine, nifti.header)
out_nifti_pm.to_filename('sensitivity_analysis_prediction_map_step%d_radius%d.nii.gz' % (step_size, radius))

a = np.copy(pos_relevance_map)
a = np.squeeze(a) # remove first subject index and last color channel again
a = np.flip(a) # flip all positions
a = np.transpose(a, (1, 2, 0)) # reorder dimensions from coronal view z*x*y back to x*y*z
new_data = np.zeros(data.shape, dtype=np.float32)
new_data[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to] = a
out_nifti_rm = nib.Nifti1Image(new_data, nifti.affine, nifti.header)
out_nifti_rm.to_filename('sensitivity_analysis_relevance_map_pos_step%d_radius%d.nii.gz' % (step_size, radius))

a = np.copy(neg_relevance_map)
a = np.squeeze(a) # remove first subject index and last color channel again
a = np.flip(a) # flip all positions
a = np.transpose(a, (1, 2, 0)) # reorder dimensions from coronal view z*x*y back to x*y*z
new_data = np.zeros(data.shape, dtype=np.float32)
new_data[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to] = a
out_nifti_rm = nib.Nifti1Image(new_data, nifti.affine, nifti.header)
out_nifti_rm.to_filename('sensitivity_analysis_relevance_map_neg_step%d_radius%d.nii.gz' % (step_size, radius))

a = np.add(pos_relevance_map, neg_relevance_map)
a = np.squeeze(a) # remove first subject index and last color channel again
a = np.flip(a) # flip all positions
a = np.transpose(a, (1, 2, 0)) # reorder dimensions from coronal view z*x*y back to x*y*z
new_data = np.zeros(data.shape, dtype=np.float32)
new_data[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to] = a
out_nifti_rm = nib.Nifti1Image(new_data, nifti.affine, nifti.header)
out_nifti_rm.to_filename('sensitivity_analysis_relevance_map_step%d_radius%d.nii.gz' % (step_size, radius))